In [ ]:
#Import all necessary libraries

import matplotlib.pyplot as plt # for plotting maps
import pandas as pd # standard python data library
import geopandas as gp # the geo-version of pandas
import numpy as np
import time
import requests
import os

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)   

wd = os.getcwd()
pdir = os.path.dirname(wd)
f2020 = os.path.join(pdir,'2020')
fcsv = os.path.join(f2020,'csv')
bg_csv = os.path.join(fcsv,'bg_cvap_2020_csv')

In [ ]:
def counties(state_fips):
    """Inputs: state fips code
    Process: Retrieves a list of counties in the given state from the Census API.  
    Outputs: A pandas dataframe of county fips codes in the state. """
    #uses the fips input into the census api
    resp = requests.get(
        "https://api.census.gov/data/2020/dec/pl"
        "?get=NAME&for=county:*&in=state:{}".format(state_fips)  #uses the fips input to locate the state
    )
    #retrieves the data as a json 
    header, *rows = resp.json()
    #county column is "county"
    county_column_index = header.index("county")
    county_fips = set(row[county_column_index] for row in rows) #sequence of counties 
    county_name_index = header.index("NAME")
    county_names = set(row[county_name_index] for row in rows)
    county_fips = np.array(list(county_fips))
    county_names = np.array(list(county_names))
    df = pd.DataFrame({'COUNTYFP20': county_fips, 'COUNTYNAMES': county_names}) #make pd dataframe of arrays
    df['COUNTY_STATE_FIPS']=state_fips + df['COUNTYFP20']
    return df #returns the fips codes of all counties

In [ ]:
def get_2020pl_data(fip, geog, y, CENSUS_API_KEY = 'ef8d7d2d71226a4e4f86b6ee741c8d8f979d6c7b'):
    """
    Inputs:
    Outputs:
    """
    HOST = "https://api.census.gov/data"
    year = "2020"
    dataset = "dec/pl"
    base_url = "/".join([HOST, year, dataset])
    # The variables we want are NAME and total population
    #get_variables = ["NAME", "P001001"]   
    data = []
    variables = ['P4_001N','P4_003N','P4_007N','P4_008N','P4_006N','P4_009N','P4_005N','P4_014N','P4_015N','P4_013N','P4_018N','P4_011N','P4_002N','P2_001N','P2_003N','P2_007N','P2_008N','P2_006N','P2_009N','P2_005N','P2_014N','P2_015N','P2_013N','P2_018N','P2_011N','P2_002N']
    new_names = ['CVAP_TOT'+y,'CVAP_NHS'+y,'CVAP_AIA'+y,'CVAP_ASN'+y,'CVAP_BLK'+y,'CVAP_NHP'+y,'CVAP_WHT'+y,'CVAP_AIW'+y,'CVAP_ASW'+y,'CVAP_BLW'+y,'CVAP_AIB'+y,'CVAP_2OM'+y,'CVAP_HSP'+y,'C_TOT'+y,'C_NHS'+y,'C_AIA'+y,'C_ASN'+y,'C_BLK'+y,'C_NHP'+y,'C_WHT'+y,'C_AIW'+y,'C_ASW'+y,'C_BLW'+y,'C_AIB'+y,'C_2OM'+y,'C_HSP'+y]
    #print(len(variables),len(new_names))
    rename_dict = dict(zip(variables,new_names))
    #for k,v in rename_dict.items():
        #print(k, v)
    counties_codes_TEST = counties(fip)
    counties_fips = counties_codes_TEST['COUNTY_STATE_FIPS']
    print('starting to collect data for ' + geog + ' ' + fip)
    for county in counties_fips:
       # for county_code in state_county_dict[fip]:
        predicates = {}
        predicates["get"] = ",".join(variables)
        if geog == 'b':
            predicates["for"] = "block:*"
        if geog == 'bg':
            predicates["for"] = "block group:*"
        predicates["in"] = "state:" + fip + "+county:" + county[2:]
        predicates["key"] = CENSUS_API_KEY
        # Write the result to a response object:
        response = requests.get(base_url, params=predicates)
        #print(response.url)
        #print(response)
        col_names = response.json()[0]        
        data = data + response.json()[1:]
        print("found data for: " + county + "!")
    print('done collecting data for', fip)
    geoids = []  # initialize geoid vector
    pop_data = pd.DataFrame(columns=col_names, data=data)
    pop_data.rename(columns=rename_dict, inplace=True)
    cols = [i for i in pop_data.columns if i not in ["NAME","state","county","tract",'block group',"block"]]
    for col in cols:
        pop_data[col]=pd.to_numeric(pop_data[col])
    for index, row in pop_data.iterrows():
        # make changes here for tracts
        if geog == 'b':
            geoid = row["state"] + row["county"] + row["tract"] + row["block"]
        if geog == 'bg':
            geoid = row["state"] + row["county"] + row["tract"] + row["block group"]
        geoids.append(geoid)
        
        
    if geog == 'b':
        pop_data["block group"] = pop_data["block"].apply(lambda x: x[0])
        pop_data["BG_GEOID"] = pop_data.apply(lambda x: x["state"] + x["county"] + x["tract"] + x["block group"], axis=1)
    pop_data["GEOID"] = geoids
    
    pivot = pop_data

    pivot['C_AIA'+y] = pivot.apply(lambda x: x['C_AIA'+y]+x['C_AIB'+y]+x['C_AIW'+y],axis=1)
    pivot['CVAP_AIA'+y] = pivot.apply(lambda x: x['CVAP_AIA'+y]+x['CVAP_AIB'+y]+x['CVAP_AIW'+y],axis=1)
    pivot['C_BLK'+y] = pivot.apply(lambda x: x['C_BLK'+y]+x['C_BLW'+y]+x['C_AIB'+y],axis=1)    
    pivot['CVAP_BLK'+y] = pivot.apply(lambda x: x['CVAP_BLK'+y]+x['CVAP_BLW'+y]+x['CVAP_AIB'+y],axis=1)
    pivot['C_ASN'+y] = pivot.apply(lambda x: x['C_ASN'+y] + x['C_ASW'+y],axis=1)
    pivot['CVAP_ASN'+y] = pivot.apply(lambda x: x['CVAP_ASN'+y] + x['CVAP_ASW'+y],axis=1)
    pivot['CVAP_2OM'+y] = pivot.apply(lambda x: x['CVAP_2OM'+y] - x['CVAP_AIB'+y] - x['CVAP_AIW'+y] - x['CVAP_BLW'+y] - x['CVAP_ASW'+y],axis=1)
    pivot['C_2OM'+y] = pivot.apply(lambda x: x['C_2OM'+y] - x['C_AIB'+y] - x['C_AIW'+y] - x['C_BLW'+y] - x['C_ASW'+y],axis=1)    
    
    return pivot

In [ ]:
def get_bg_row(bg_geoid, col, bg_df, iteration = 0, disp = False):
    bg_df['GEOID'] = bg_df['GEOID'].astype(str)
    bg_row = bg_df[bg_df['GEOID']==bg_geoid].copy()
    #display(bg_row)
    if disp == True:
        print('GEOID is', bg_geoid)
        print('Length of DF: ', len(bg_row))
        display(bg_row)
    val = list(bg_row[col])

    try:
        val = val[0]
    except:
        print('ERROR')
        print('BG GEOID: ', bg_geoid)
    return int(val)

In [ ]:
def assign_ab(fips):
    values = ['01','02','04','05','06','08','09','10','12','13','15','16','17','18','19','20','21','22','23',
                  '24','25','26','27','28','29','30','31','32','33','34','35','36','37','38','39','40','41','42','44','45','46',
                  '47','48','49','50','51','53','54','55','56']
    keys = ['al','ak','az','ar','ca','co','ct','de','fl','ga','hi','id','il','in','ia','ks','ky','la','me','md','ma','mi','mn','ms','mo','mt','ne','nv','nh','nj','nm','ny','nc','nd','oh','ok','or','pa','ri','sc','sd','tn','tx','ut','vt','va','wa','wv','wi','wy']
    dictionary = dict(zip(keys,values))
    state_ab = ''
    for key, value in dictionary.items(): 
        if value == fips: 
            state_ab=key
    return state_ab

In [ ]:
def mod_mismatch(b_df,cvap,i,y):
    cur_sum = b_df[i].sum()
    cvap_sum = cvap[i].sum()
    bg_ids = list(b_df['BG_GEOID'].unique())
    if int(cvap_sum)!= 0:
        rem = cvap_sum-cur_sum
        for bg_id in bg_ids:
            b_sub = b_df[b_df['BG_GEOID']==bg_id].copy()
            cvap_sub = cvap[cvap['GEOID']==bg_id].copy()
            b_sub_tot_i = b_sub[i].sum()
            cvap_sub_tot_i = cvap_sub[i].sum()
            bg_rem = cvap_sub_tot_i-b_sub_tot_i
            if int(bg_rem) != 0:
                if 'C_' in bg_id:
                    c_tot_sum = int(cvap_sub['C_TOT'+y].sum())
                    if int(c_tot_sum)!= 0:
                        if i!='C_TOT'+y:
                            b_sub[i] = b_sub['C_TOT'+y].apply(lambda x: float(x/c_tot_sum))
                            b_sub[i] = b_sub[i].apply(lambda x: x*bg_rem)
                        else:
                            val = c_tot_sum/len(b_sub)
                            b_sub[i] = val
                    else:
                        b_sub[i] = float(0)
                else:
                    cvap_tot_sum = int(cvap_sub['CVAP_TOT'+y].sum())
                    if int(cvap_tot_sum)!= 0:
                        if i!='CVAP_TOT'+y:
                            b_sub[i] = b_sub['CVAP_TOT'+y].apply(lambda x: float(x/cvap_tot_sum))
                            b_sub[i] = b_sub[i].apply(lambda x: x*bg_rem)
                        else:
                            val = float(cvap_tot_sum/len(b_sub))
                            b_sub[i] = val
                    else:
                        b_sub[i] = float(0)
                b_df = b_df[b_df['BG_GEOID']!=bg_id].copy()
                b_df = pd.concat([b_sub,b_df])
            else:
                continue
    else:
        b_df[i] = float(0)
    new_sum = b_df[i].sum()
    for bg_id in bg_ids:
        b_sub = b_df[b_df['BG_GEOID']==bg_id].copy()
        cvap_sub = cvap[cvap['GEOID']==bg_id].copy()
        b_sub_tot_i = b_sub[i].sum()
        cvap_sub_tot_i = cvap_sub[i].sum()
        if round(cvap_sub_tot_i)!= round(b_sub_tot_i):
            print('***THERE IS STILL A MISMATCH***')
            print('CVAP TOTAL FOR BG ', str(bg_id), ' COLUMN ', i, ' IS: ', str(cvap_sub_tot_i))
            print('NEW TOTAL FOR BG ', str(bg_id), ' COLUMN ', i, ' IS: ', str(b_sub_tot_i))
    if round(cvap_sum)!=round(new_sum):
        print('SUMS ARE NOT EQUAL')
        print('CVAP SUM: ', str(cvap_sum))
        print('NEW SUM: ', str(new_sum))
    return b_df

In [ ]:
def set_types(wd = os.getcwd()):
    ak_csv = pd.read_csv(os.path.join(os.path.join(bg_csv,'ak_cvap_2020_bg'),'ak_cvap_2020_bg.csv'))
    cvap_cols = list(ak_csv.columns)
    type_list = []
    for i in cvap_cols:
        if i.startswith('CVAP'):
            type_list.append("int")
        if i.startswith('C_'):
            type_list.append("int")
        else:
            type_list.append("object")

    dtype_dict = dict(zip(cvap_cols,type_list))
    return dtype_dict

In [ ]:
def get_ratio_table(fips,year):
    y = year[-2:]
    dtype_dict = set_types()
    b = get_2020pl_data(fips,'b',y)
    bg = b.groupby('BG_GEOID').sum()
    bg.reset_index(inplace=True)
    bg['GEOID'] = bg['BG_GEOID'].astype(str)
    sa = assign_ab(fips)
    print('On state: ', sa.upper())
    wd = os.getcwd()
    cvap_folders  = bg_csv
    for i in os.listdir(cvap_folders):
        if '.zip' not in i:
            if i.startswith(sa):
                folder_path = os.path.join(cvap_folders,i)
                for j in os.listdir(folder_path):
                    if j.endswith('.csv'):
                        csv_path = os.path.join(folder_path,j)
    cvap = pd.read_csv(csv_path,dtype=dtype_dict)
    if len(bg)!=len(cvap):
        print('LENGHTS OF BG AND CVAP DO NOT EQUAL')
        return 
    cvap['GEOID'] = cvap['GEOID'].astype(str)
    no_data_cols = ['GEOID','BG_GEOID','state','county','tract','block', 'block group','NAME']
    start_process_time = time.time()
    for i in b.columns:
        if i not in no_data_cols:
            print('Starting column ', i, '...')
            col_start = time.time()
            print('STARTING RATIOS!')
            b[i] = b.apply(lambda x: float(int((x[i]))/int(get_bg_row(x['BG_GEOID'], i, bg))) * get_bg_row(x['BG_GEOID'],i,cvap) if get_bg_row(x['BG_GEOID'], i, bg) != 0 else 0,axis=1)
            if round(b[i].sum())!=round(cvap[i].sum()):
                print('PRIOR TO MISMATCH PROCESSING BLOCK TOTAL IS: ', str(b[i].sum()))
                print('PRIOR TO MISMATCH PROCESSING CVAP TOTAL IS: ', str(cvap[i].sum()))
                b = mod_mismatch(b,cvap,i,y)
            col_end = time.time()
            diff_sec = col_end-col_start
            diff_min = diff_sec/60
            print('TIME TAKEN FOR ', i, ': ', str(diff_min), ' MINUTES.')
        else:
            continue
    cols_to_drop = []
    for i in list(b.columns):
        if i.startswith('floor'):
            cols_to_drop.append(i)
        elif i.startswith('re'):
            cols_to_drop.append(i)
        else:
            continue
    b.drop(columns=cols_to_drop,inplace=True)
    for i in b.columns:
        if i not in no_data_cols:
            #print('\nCVAP ORIGINAL TOTAL FOR ',i,':',cvap[i].sum())
            #print('NEW ALLOCATED BLOCK TOTALS FOR',i,':',b[i].sum())
            print('\n')
            if round(cvap[i].sum())==round(b[i].sum()):
                print(i, ' IS EQUAL.')
                continue
            else:
                print('NOT EQUAL FOR ',i)
                print('CVAP ',i, ' IS: ',str(cvap[i].sum()))
                print('NEW BLOCKS ', i , ' IS: ', str(b[i].sum()))
    #display(b.head())
    end_process_time = time.time()
    process_time_sec = end_process_time - start_process_time
    process_time_min = process_time_sec/60
    process_time_hr = process_time_min/60
    print(sa.upper(), ' TOTAL COLUMN PROCESSING TIME TO GET TO 2020 BLOCKS: ', str(process_time_hr), ' HOURS.')
    return b

In [ ]:
def disag(fips,year):
    disag = get_ratio_table(fips,year)
    disag_folder = os.path.join(os.getcwd(),'DISAG_CSV')
    if not os.path.exists(disag_folder):
        os.mkdir(disag_folder)
    sa = assign_ab(fips)
    name = '_'.join([sa.lower(),'cvap',year,'2020','b.csv'])
    extract_path = os.path.join(disag_folder,name)
    disag.to_csv(extract_path,index=False)#Just added, need to re-run with this
    return disag

In [ ]:
disag_list = ['01','02','04','05','08','09','10','12','13','15','16','17','18','19','20','21','22','23',
                  '24','25','26','27','28','29','30','31','32','33','34','35','36','37','38','39','40','41','42','44','45','46',
                  '47','48','49','50','51','53','54','55','56','06']

In [ ]:
def run_disag(fips_list = disag_list,years_list = ['2020']):
    datasets = []
    abbrv = []
    for y in years_list:
        for state in disag_list:
            sa = assign_ab(state)
            print('STARTING: ', sa.upper(), ' ', y)
            start_time = time.time()
            disag_dataset = disag(state,y)
            datasets.append(disag_dataset)
            abbrv.append(sa)
            print('FINISHED: ', sa.upper(), ' ', y)
            end_time = time.time()
            diff_sec = end_time - start_time
            diff_min = diff_sec/60
            diff_hr = diff_min/60
            print('TOTAL TIME TO RUN FOR ', sa.upper(),' ', y, ': ', str(diff_hr))
    datasets_dict = dict(zip(abbrv,datasets))
    return datasets_dict

In [ ]:
datasets = run_disag()

In [ ]:
def view_datasets(dataset_dict = datasets):
    for k,v in dataset_dict.items():
        print(k, ' IS DONE.')
view_datasets()